In [1]:
# Dependencies installation
!pip -q install --upgrade pip
!pip -q install accelerate peft bitsandbytes transformers trl datasets torch torchvision torchaudio tokenizers

# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel, PeftConfig
from trl import SFTTrainer
from datasets import load_dataset

# Clear CUDA cache
torch.cuda.empty_cache()

# Base model and dataset configuration
base_model = "NousResearch/Llama-2-7b-chat-hf"
# my_dataset = "Leonardorm7/PP"
my_dataset = "Leonardorm7/PPAPI"
new_model = "llama-2-7b-PowerPoint"

# Quantization configuration
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Load pretrained model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto"
)
model.config.use_cache = False

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# PEFT configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Load dataset
dataset = load_dataset(my_dataset)

def preprocessing(sample):
    system_prompt_template = """<s>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: <<user_question>>
### Response:
<<user_response>>
</s>
"""
    user_message = sample['Input']
    user_response = sample['Python code']
    prompt_template = system_prompt_template.replace("<<user_question>>", user_message).replace("<<user_response>>", user_response)
    return {"train": prompt_template}

training_dataset = dataset.map(preprocessing)
train_dataset = training_dataset['train'].map(lambda x: x, batched=True)
test_dataset = training_dataset['train'].map(lambda x: x, batched=True)

# Training configuration
training_params = TrainingArguments(
    output_dir="./PPLLM",
    evaluation_strategy='steps',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    dataset_text_field="train",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

# Save the model and tokenizer
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

print('Model Fine-Tuned and Saved')


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Model Fine-Tuned and Saved


# Trained model test


In [2]:

base_model_name = "NousResearch/Llama-2-7b-chat-hf"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
config = PeftConfig.from_pretrained(new_model)
model = PeftModel.from_pretrained(base_model, new_model)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
 # Prepare the instruction

instruction='Create a 5-slide presentation with yellow background, each slide with Slide Title in Calibri size 45 and a bullet list containing four points in Calibri size 20'
prompt = f"<s>\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction: {instruction}\n### Response:\n"

# Generate response without few-shot
response_without_few_shot = generator(prompt, max_length=500, num_return_sequences=1, temperature=0.7)

# Print generated response
print(response_without_few_shot[0]['generated_text'])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

<s>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: Create a 5-slide presentation with yellow background, each slide with Slide Title in Calibri size 45 and a bullet list containing four points in Calibri size 20
### Response:

Slide 1: Introduction

* Title: Introduction to the Importance of Sustainability in Business
* Bullet points:
	+ Definition of sustainability and its importance in business
	+ Benefits of sustainability for businesses and society
	+ Challenges of implementing sustainability in businesses
	+ Importance of stakeholder engagement in sustainability initiatives

Slide 2: Environmental Sustainability

* Title: Environmental Sustainability in Business
* Bullet points:
	+ Reducing carbon footprint and energy consumption
	+ Implementing recycling and waste reduction programs
	+ Sourcing materials sustainably
	+ Measuring and reporting environmental performance

Slide 3: Social Sustainability

* Ti

# Few shot Answer

In [3]:
   
# Examples
few_shot_examples = [
    {
        "instruction": "Create a 4-slide presentation with light green background, each slide with Slide Title in Times New Roman size 40 and a bullet list containing two points in Times New Roman size 30",
        "response": """
api.create_presentation('Light Green Slides')

titles = ["Slide 1", "Slide 2", "Slide 3", "Slide 4"]
bullet_points = [
    ["Point 1", "Point 2"],
    ["Item A", "Item B"],
    ["Fact 1", "Fact 2"],
    ["Detail X", "Detail Y"]
]

for i in range(4):
    api.add_slide(layout='title and content')
    api.change_background_color(i + 1, (144, 238, 144))
    api.add_text_to_slide(i + 1, titles[i], placeholder=0)
    for point in bullet_points[i]:
        api.add_bullet_point(i + 1, point)
    api.change_font(i + 1, 0, 'Times New Roman')
    api.change_font_size(i + 1, 0, 40)
    api.change_font(i + 1, 1, 'Times New Roman')
    api.change_font_size(i + 1, 1, 30)
api.save_presentation('light_green_slides.pptx')
        """
    },
    {
        "instruction": "Create a 3-slide presentation with blue background, each slide with Slide Title in Arial size 35 and a bullet list containing three points in Arial size 25",
        "response": """
api.create_presentation('Blue Slides')

titles = ["Slide 1", "Slide 2", "Slide 3"]
bullet_points = [
    ["Point A", "Point B", "Point C"],
    ["Item 1", "Item 2", "Item 3"],
    ["Detail X", "Detail Y", "Detail Z"]
]

for i in range(3):
    api.add_slide(layout='title and content')
    api.change_background_color(i + 1, (0, 0, 255))
    api.add_text_to_slide(i + 1, titles[i], placeholder=0)
    for point in bullet_points[i]:
        api.add_bullet_point(i + 1, point)
    api.change_font(i + 1, 0, 'Arial')
    api.change_font_size(i + 1, 0, 35)
    api.change_font(i + 1, 1, 'Arial')
    api.change_font_size(i + 1, 1, 25)
api.save_presentation('blue_slides.pptx')
        """
    }
]

# Function to create the few-shot prompt
def create_few_shot_prompt(instruction, examples):
    prompt = "Below are some examples of how to create presentations:\n\n"
    for example in examples:
        prompt += f"### Instruction:\n{example['instruction']}\n### Response:\n{example['response']}\n\n"
    prompt += "### Instruction:\n" + instruction + "\n### Response:\n"
    return prompt

# Generate the few-shot prompt
few_shot_prompt = create_few_shot_prompt(instruction, few_shot_examples)

# Tokenize the few-shot prompt
inputs = tokenizer(few_shot_prompt, return_tensors="pt")

# Generate the response with few-shot
output = model.generate(**inputs, max_length=2000, num_return_sequences=1, temperature=0.7)

# Decode the output
response_with_few_shot = tokenizer.decode(output[0], skip_special_tokens=True)
print("\nAnswer with few-shot:")
print(response_with_few_shot)




Answer with few-shot:
Below are some examples of how to create presentations:

### Instruction:
Create a 4-slide presentation with light green background, each slide with Slide Title in Times New Roman size 40 and a bullet list containing two points in Times New Roman size 30
### Response:

api.create_presentation('Light Green Slides')

titles = ["Slide 1", "Slide 2", "Slide 3", "Slide 4"]
bullet_points = [
    ["Point 1", "Point 2"],
    ["Item A", "Item B"],
    ["Fact 1", "Fact 2"],
    ["Detail X", "Detail Y"]
]

for i in range(4):
    api.add_slide(layout='title and content')
    api.change_background_color(i + 1, (144, 238, 144))
    api.add_text_to_slide(i + 1, titles[i], placeholder=0)
    for point in bullet_points[i]:
        api.add_bullet_point(i + 1, point)
    api.change_font(i + 1, 0, 'Times New Roman')
    api.change_font_size(i + 1, 0, 40)
    api.change_font(i + 1, 1, 'Times New Roman')
    api.change_font_size(i + 1, 1, 30)
api.save_presentation('light_green_slides.